In [ ]:
# from keras import Input
# from sklearn import metrics
# from tensorflow.keras.callbacks import EarlyStopping
# from keras import optimizers
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers, callbacks
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, Dropout, Dense, Flatten, add
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
!pip install keras-tuner

import keras_tuner as kt
from keras_tuner import Hyperband, HyperModel, HyperParameters
hp = HyperParameters()

     |████████████████████████████████| 102kB 5.3MB/s 
  Created wheel for kt-legacy: filename=kt_legacy-1.0.3-cp37-none-any.whl size=9569 sha256=92b73ce8c839d4293f623aec1872f0fe3c09ede5fecad91c00166e3f1f5ed3b7
  Stored in directory: /root/.cache/pip/wheels/95/99/a8/6b116b0f69be60cc475d1cd36680f3a09f284d86655bb99d93
Successfully built kt-legacy


In [ ]:
# def build_model(hp):
#     inputs = Input(shape=(32,32,3))
#     x = inputs
#     for i in range(hp.Int("conv_blocks", min_value=3, max_value=5, step=1, default=3)):
#         filters = hp.Int("filters_" + str(i), 32, 256, step=32)
#         for _ in range(2):
#             x = Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
#             x = BatchNormalization()(x)
#             x = Activation('relu')(x)
#         if hp.Choice("pooling_" + str(i), ["avg", "max"]) == 'max':
#             x = MaxPooling2D()(x)
#         else:
#             x = AveragePooling2D()(x)
#     x = GlobalAveragePooling2D()(x)
#     x = Dense(hp.Int("hidden_size", min_value=30, max_value=100, step=10, default=50),
#         Activation('relu'))(x)
#     x = Dropout(hp.Float("dropout", min_value=0, max_value=0.5, step=0.1, default=0.5))(x)
#     outputs = Dense(10, activation='softmax')(x)
#     model = Model(inputs, outputs)
#     model.compile(
#         optimizer = tf.keras.optimizers.Adam(
#             hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
#         loss = 'categorical_crossentropy',
#         metrics=['accuracy'],
#     )

#     return model

In [ ]:
# ResNet9 - Subclassing 

class ResNet9(kt.HyperModel):
    def Residual_bottleneck(inputs, f, stride = 1, bn_eps = 2e-5, bn_mom = 0.9, reg = 0.0005):
        
        # step 1. (bn - relu - conv 1x1)
        bn_1 = BatchNormalization(epsilon = bn_eps, momentum = bn_mom)(inputs)
        act_1 = Activation("relu")(bn_1)
        conv_1 = Conv2D(int(f * 0.25), (1, 1), use_bias = False, kernel_regularizer = regularizers.l2(reg))(act_1)

        # step 2. (bn - relu - conv 3x3)
        bn_2 = BatchNormalization(epsilon = bn_eps, momentum = bn_mom)(conv_1)
        act_2 = Activation("relu")(bn_1)
        conv_2 = Conv2D(int(f * 0.25), (3, 3), strides = stride, padding = "same", use_bias = False, 
                        kernel_regularizer = regularizers.l2(reg))(act_2)

        # step 3. (bn - relu - conv 1x1)
        bn_3 = BatchNormalization(epsilon = bn_eps, momentum = bn_mom)(conv_2)
        act_3 = Activation("relu")(bn_3)
        conv_3 = Conv2D(f, (1, 1), use_bias = False, kernel_regularizer = regularizers.l2(reg))(act_3)

        # conv skip
        conv_skip = Conv2D(f, (1, 1), strides = stride, use_bias = False, kernel_regularizer = regularizers.l2(reg))(act_1)

        shortcut = inputs
        if stride == 2:
            shortcut = conv_skip
      
        output = add([conv_3, shortcut])

        return output

    def build(self, hp, inputs = (32, 32, 3), bn_eps = 2e-5, bn_mom = 0.9):

        # filters = (64, 64, 128, 256)
        input = Input(inputs)
        hyper_f1 = hp.Int('filters_1_2', 16, 64, step = 16, default = 16)
        hyper_f2 = hyper_f1
        hyper_f3 = hp.Int('filters_3', 32, 128, step = 16, default = 32)
        hyper_f4 = hp.Int('filters_4', 64, 256, step = 16, default = 64)

        hyper_reg = hp.Float('regularizer_L2', 0.0001, 0.0005, step = 1e-4)

        x = BatchNormalization(epsilon = bn_eps, momentum = bn_mom)(input)
        x = Conv2D(hyper_f1, (3, 3), padding = 'same', use_bias = False, kernel_regularizer = regularizers.l2(hyper_reg))(x)
        
        # hint:
        # stage[0] -> filters[1] * 9 (stride = 1)
        # stage[1] -> filters[2] * 1 (stride = 2) + filters[2] * 8 (stride = 1)
        # stage[2] -> filters[3] * 1 (stride = 2) + filters[3] * 8 (stride = 1)
        # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ #
        
        # for s in range(0, len(stages)):
        #     stride = 1 if s == 0 else 2
        #     x = ResNet9.Residual_bottleneck(x, filters[s + 1], stride, bn_eps = bn_eps, 
        #                                     bn_mom = bn_mom, reduce = True)
        #     for i in range(0, stages[s] - 1):
        #         x = ResNet9.Residual_bottleneck(x, filters[s + 1], (1, 1), bn_eps = bn_eps,
        #                                         bn_mom = bn_mom)

        # loop over the number of stages
        # s1, s2, s3 = stages
        stage = hp.Int('conv_blocks', 9, 9, default = 9)
        prev_filters = hyper_f2

        for i in [hyper_f2] * stage + [hyper_f3] * stage + [hyper_f4] * stage:
            stride = 1 if i == prev_filters else 2
            x = ResNet9.Residual_bottleneck(x, i, stride, bn_eps = bn_eps, bn_mom = bn_mom)
            prev_filters = i

        # bn -> act -> pool
        x = BatchNormalization(epsilon = bn_eps, momentum = bn_mom)(x)
        x = Activation('relu')(x)
        x = AveragePooling2D((8, 8))(x)

        # flatten -> dense -> softmax
        x = Flatten()(x)
        x = Dense(10, kernel_regularizer = regularizers.l2(hyper_reg))(x)
        x = Activation('softmax')(x)

        model = Model(input, x)

        # compile the model
        print("[INFO] Compiling Model...")
        optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1, clipvalue = 0.1)  # default: lr = 0.01, mom = 0.99  # decay

        model.compile(optimizer = optimizer, 
                      loss = 'categorical_crossentropy', metrics = ['accuracy'])
        return model

In [ ]:
# Learning_rate_decay

def lr_decay(epoch):
    base_lr = hp.Float('learning_rate', 0.1, 0.1, step = 0, default = 0.1)   # ResNet9: 0.1
    pow = 1.0
    lr_decay = base_lr * (1 - epoch/EPOCHS)**pow

    return lr_decay

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os 

DRIVE = '/content/drive/MyDrive/'

ML_PATH = "大伯 - ML_Code/"

if os.path.isdir(os.path.join(DRIVE, 'Colab Notebooks/')):
    MODEL_PATH = os.path.join(DRIVE, 'Colab Notebooks/', ML_PATH)   # drive_1
else:
    MODEL_PATH = os.path.join(DRIVE, ML_PATH)                       # drive_2

os.listdir(MODEL_PATH)

In [ ]:
hyper_dir = os.path.join(MODEL_PATH, 'hyper_dir/')

tuner = kt.Hyperband(
    ResNet9(),
    objective='val_accuracy',
    max_epochs=30,
    directory = hyper_dir,
    project_name='hyper_record',
    hyperband_iterations=2)

[INFO] Compiling Model...


In [ ]:
# Golab GPU devices status

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
! nvidia-smi 

Num GPUs Available:  1
Fri Jul 16 09:21:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    26W /  70W |    224MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                 

In [ ]:
(train_X, train_Y), (test_X, test_Y) = datasets.cifar10.load_data()

print(train_X.shape)
print(test_X.shape)

# Normalize and Zero-Centered
def standardize(X, Y):
    X, Y = X.astype('float32') / 255., to_categorical(Y, 10)
    mean = np.mean(X, axis = 0)
    X -= mean
    return X, Y

train_X, train_Y = standardize(train_X, train_Y)
test_X, test_Y = standardize(test_X, test_Y)

EPOCHS = 50
# lr_scheduling
lr_decay = callbacks.LearningRateScheduler(lr_decay)

# stop training early after reaching a certain value for the validation loss
stop_early = callbacks.EarlyStopping(monitor='val_loss',patience=5)

# Run the hyperparameters search
tuner.search(train_X, train_Y, epochs=EPOCHS, validation_split=0.2,
            callbacks = [stop_early, lr_decay])

Trial 41 Complete [00h 02m 28s]
val_accuracy: 0.6834999918937683

Best val_accuracy So Far: 0.7060999870300293
Total elapsed time: 01h 21m 19s

Search: Running Trial #42

Hyperparameter    |Value             |Best Value So Far 
filters_1_2       |48                |48                
filters_3         |96                |112               
filters_4         |112               |240               
regularizer_L2    |0.0003            |0.0001            
conv_blocks       |9                 |9                 
tuner/epochs      |4                 |4                 
tuner/initial_e...|2                 |2                 
tuner/bracket     |3                 |3                 
tuner/round       |1                 |1                 
tuner/trial_id    |30c669d22a8fabf...|8f2727cc90ea98b...

[INFO] Compiling Model...
Epoch 3/4
1250/1250 [==============================] - 51s 37ms/step - loss: 2.4303 - accuracy: 0.4446 - val_loss: 2.1232 - val_accuracy: 0.5281
Epoch 4/4
 567/1250 [=========

KeyboardInterrupt: ignored

In [ ]:
tuner.results_summary()

Results summary
Results in /content/drive/MyDrive/大伯 - Cifar10_Code/hyper_dir/hyper_record
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
filters_1_2: 48
filters_3: 112
filters_4: 240
regularizer_L2: 0.0001
conv_blocks: 9
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 3
tuner/round: 1
tuner/trial_id: 8f2727cc90ea98b003636b702192e4e2
Score: 0.7060999870300293
Trial summary
Hyperparameters:
filters_1_2: 48
filters_3: 128
filters_4: 160
regularizer_L2: 0.0002
conv_blocks: 9
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 3
tuner/round: 1
tuner/trial_id: 162c10b1f959c322c2e532434c1c9aff
Score: 0.6959999799728394
Trial summary
Hyperparameters:
filters_1_2: 48
filters_3: 112
filters_4: 240
regularizer_L2: 0.0001
conv_blocks: 9
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.6920999884605408
Trial summary
Hyperparameters:
filters_1_2: 32
filters_3: 128
filters_4: 176
regularizer_L2: 0.0002
conv

In [ ]:
best_model = tuner.get_best_models(1)[0]

best_hypers = tuner.get_best_hyperparameters(1)[0]
print(f'''best conv_blocks: {best_hypers.get('conv_blocks')}''',
      f'''\nbest filters_1_2: {best_hypers.get('filters_1_2')}''',
      f'''\nbest filters_3: {best_hypers.get('filters_3')}''',
      f'''\nbest filters_4: {best_hypers.get('filters_4')}''',
      f'''\nbest regularizer_L2: {best_hypers.get('regularizer_L2')}''')

[INFO] Compiling Model...
best conv_blocks: 9 
best filters_1_2: 48 
best filters_3: 112 
best filters_4: 240 
best regularizer_L2: 0.0001


In [ ]:
# Construct the image generator for Data Augmentation
print("[INFO] Using Data Augmentation...")
data_aug = ImageDataGenerator(width_shift_range = 0.1, height_shift_range = 0.1,
                              horizontal_flip = True)   # rotation_range = 10 

[INFO] Using Data Augmentation...


In [ ]:
Batch_Size = 128
history = best_model.fit(data_aug.flow(train_X, train_Y, Batch_Size),
                         validation_data = (test_X, test_Y),
                         steps_per_epoch = len(train_X) // Batch_Size, epochs = 100,
                         callbacks = [stop_early, lr_decay], verbose = 1)

val_acc = history.history['val_accuracy']

best_epoch = val_acc[max(val_acc)] + 1
print(f'best epoch: {best_epoch}')

Epoch 1/100
390/390 [==============================] - 67s 157ms/step - loss: 1.7206 - accuracy: 0.7503 - val_loss: 1.7344 - val_accuracy: 0.7369
Epoch 2/100
150/390 [==========>...................] - ETA: 33s - loss: 1.6090 - accuracy: 0.7739

KeyboardInterrupt: ignored

In [ ]:
# Re-instantiate hypermodel
model = tuner.hypermodel.build(best_hypers)

# Retrain the model
model.fit(train_X, train_Y, epochs = best_epoch, validation_split = 0.2)

In [ ]:
# Predict Model

eval_loss, eval_acc = model.evaluate(test_X, test_Y)
print(f'test_loss: {eval_loss:.4f}, \ntest_acc: {eval_acc:.4f}')